In [4]:
import pandas as pd  # Used for working with tabular data (like Excel or CSV files)
import ast  # Used to safely convert strings that look like Python data structures into real Python objects
import logging  # For tracking code execution and debugging

In [5]:
# -----------------------
# Configure Logging
# -----------------------
logging.basicConfig(
    level=logging.INFO,  # Use logging.DEBUG for more detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("scheme_mapping_debug.log"),
        logging.StreamHandler()
    ]
)


In [6]:
def safe_eval(val):
    """
    Safely evaluate string representations of Python lists or objects.
    """
    try:
        return ast.literal_eval(val) if isinstance(val, str) and val.startswith("[") else val
    except Exception:
        logging.warning(f"Failed to evaluate: {val}")
        return val


In [7]:
def load_and_clean_data(scheme_file, recommendation_file):
    """
    Load scheme and recommendation CSVs and clean required columns.
    """
    try:
        logging.info("Loading data files...")
        df_scheme = pd.read_csv(scheme_file)
        df_reco = pd.read_csv(recommendation_file)
        logging.info(f"Scheme mapping shape: {df_scheme.shape}")
        logging.info(f"Recommendations shape: {df_reco.shape}")
    except Exception:
        logging.critical("Failed to load input CSVs.", exc_info=True)
        raise
# Safely evaluate list-like strings
    logging.info("Converting stringified lists to actual lists...")
    df_scheme["Partner_id"] = df_scheme["Partner_id"].apply(safe_eval)
    df_reco["Recommended_Products"] = df_reco["Recommended_Products"].apply(safe_eval)
    df_reco["Similarity_Scores"] = df_reco["Similarity_Scores"].apply(safe_eval)

    return df_scheme, df_reco

In [8]:
def map_schemes_to_recommendations(df_scheme, df_reco):
    """
    Map top 3 schemes to each recommended product for each partner.
    """
    results = []

    logging.info("Mapping schemes to recommended products...")
    for _, row in df_reco.iterrows():
        partner_id = row["Partner_id"]
        for product, score in zip(row["Recommended_Products"], row["Similarity_Scores"]):
            # Look up the scheme mapping for the given product
            schemes = df_scheme[df_scheme["Product_id"] == product][["Scheme_1", "Scheme_2", "Scheme_3"]]
            scheme_1, scheme_2, scheme_3 = (
                schemes.iloc[0].fillna("Not Available").values
                if not schemes.empty
                else ("Not Available", "Not Available", "Not Available")
            )
            results.append([partner_id, product, score, scheme_1, scheme_2, scheme_3])
    df_final = pd.DataFrame(results, columns=[
        "Partner_id", "Product_id", "Similarity_Scores", "Scheme_1", "Scheme_2", "Scheme_3"
    ])
    logging.info(f"Generated final mapping with shape: {df_final.shape}")
    return df_final

In [9]:
def save_final_mapping(df_final, output_file="Final_Partner_Product_Schemes.csv"):
    """
    Save final mapped data to CSV.
    """
    try:
        df_final.to_csv(output_file, index=False)
        logging.info(f"Final partner-product-scheme mapping saved to {output_file}")
    except Exception:
        logging.critical("Failed to save output CSV.", exc_info=True)
        raise


In [10]:
def run_scheme_mapping_pipeline(scheme_file, recommendation_file):
    """
    Main pipeline to map top schemes to recommended products for each partner.
    """
    logging.info("Starting scheme recommendation mapping pipeline...")

    df_scheme, df_reco = load_and_clean_data(scheme_file, recommendation_file)
    df_final = map_schemes_to_recommendations(df_scheme, df_reco)
    save_final_mapping(df_final)

    logging.info("Pipeline executed successfully.")

# ----------------------------------
# Run the pipeline
# ----------------------------------
if __name__ == "__main__":
    scheme_file_path = "Top_Optimized_Schemes_with_LP.csv"
    recommendation_file_path = "Partner_Product_Recommendations.csv"
    run_scheme_mapping_pipeline(scheme_file_path, recommendation_file_path)

2025-04-01 12:14:17,014 - INFO - Starting scheme recommendation mapping pipeline...
2025-04-01 12:14:17,016 - INFO - Loading data files...
2025-04-01 12:14:17,023 - INFO - Scheme mapping shape: (15, 5)
2025-04-01 12:14:17,024 - INFO - Recommendations shape: (200, 3)
2025-04-01 12:14:17,025 - INFO - Converting stringified lists to actual lists...
2025-04-01 12:14:17,077 - INFO - Mapping schemes to recommended products...
2025-04-01 12:14:19,174 - INFO - Generated final mapping with shape: (600, 6)
2025-04-01 12:14:19,188 - INFO - Final partner-product-scheme mapping saved to Final_Partner_Product_Schemes.csv
2025-04-01 12:14:19,189 - INFO - Pipeline executed successfully.
